Name: Ziwei Liu
Student ID: 3766789

In [1]:
import os
import zipfile
import torch

import requests
from torch.utils.data import DataLoader
from torchvision import transforms as T
from torchvision.datasets import CIFAR10
from tqdm import tqdm
import torch.nn as nn

In [2]:
from examples.models.resnet import resnet18

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [3]:
threads = 16
torch.set_num_threads(threads)

#maybe better performance (Jupyter)
#%env OMP_PLACES=cores
#%env OMP_PROC_BIND=close
#%env OMP_WAIT_POLICY=active

#WSL
os.environ['OMP_PLACES'] = 'cores'
os.environ['OMP_PROC_BIND'] = 'close'
os.environ['OMP_WAIT_POLICY'] = 'active'

In [4]:
#axx_mult = 'mul8s_acc'


In [5]:
axx_mult = 'exact'
model = resnet18(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation

Using /root/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_exact/build.ninja...
Building extension module PyInit_conv2d_exact...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module PyInit_conv2d_exact...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_exact, skipping build step...
Loading extension module PyInit_conv2d_exact...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_exact, skipping build step...
Loading extension module PyInit_conv2d_exact...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_exact, skipping build step...
Loading exten

ResNet(
  (conv1): AdaPT_Conv2d(
    3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
    (quantizer): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
    (quantizer_w): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): AdaPT_Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (quantizer): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
        (quantizer_w): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
      )
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(

In [6]:
def val_dataloader(mean = (0.4914, 0.4822, 0.4465), std = (0.2471, 0.2435, 0.2616)):

    transform = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean, std),
        ]
    )
    dataset = CIFAR10(root="datasets/cifar10_data", train=False, download=True, transform=transform)
    dataloader = DataLoader(
        dataset,
        batch_size=1024,
        num_workers=0,
        drop_last=True,
        pin_memory=False,
    )
    return dataloader

transform = T.Compose(
        [
            T.RandomCrop(32, padding=4),
            T.RandomHorizontalFlip(),
            T.ToTensor(),
            T.Normalize(mean = (0.4914, 0.4822, 0.4465), std = (0.2471, 0.2435, 0.2616)),
        ]
    )
dataset = CIFAR10(root="datasets/cifar10_data", train=True, download=True, transform=transform)

evens = list(range(0, len(dataset), 10))
trainset_1 = torch.utils.data.Subset(dataset, evens)

data = val_dataloader()

# data_t is used for calibration purposes and is a subset of train-set
data_t = DataLoader(trainset_1, batch_size=128,
                                            shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib

def collect_stats(model, data_loader, num_batches):
     """Feed data to the network and collect statistic"""

     # Enable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.disable_quant()
                 module.enable_calib()
             else:
                 module.disable()

     for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
         model(image.cpu())
         if i >= num_batches:
             break

     # Disable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.enable_quant()
                 module.disable_calib()
             else:
                 module.enable()

def compute_amax(model, **kwargs):
 # Load calib result
 for name, module in model.named_modules():
     if isinstance(module, quant_nn.TensorQuantizer):
         if module._calibrator is not None:
             if isinstance(module._calibrator, calib.MaxCalibrator):
                 module.load_calib_amax()
             else:
                 module.load_calib_amax(**kwargs)
         print(F"{name:40}: {module}")
 model.cpu()

# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)

100%|██████████| 2/2 [00:01<00:00,  1.19it/s]
W0222 10:17:07.442359 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:07.442923 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:07.443422 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:07.443890 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:07.444278 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:07.444668 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:07.445066 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:07.445451 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:07.446092 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:07.446593 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:07.447041 140419826077888 tensor

conv1.quantizer                         : TensorQuantizer(8bit per-tensor amax=2.1255 calibrator=HistogramCalibrator quant)
conv1.quantizer_w                       : TensorQuantizer(8bit per-tensor amax=0.1513 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.6266 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0620 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer                : TensorQuantizer(8bit per-tensor amax=0.3473 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0380 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.5064 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0354 calibrator=HistogramCalibrator quant)
layer1.1

In [9]:
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

100%|██████████| 9/9 [00:17<00:00,  1.96s/it]

17.63174046576023
Accuracy of the network on the 10000 test images: 93.1424 %


In [8]:
axx_mult = 'appro1'
model = resnet18(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation
# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)
    
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

# Retraining for 15 epochs
from adapt.references.classification.train import evaluate, train_one_epoch, load_data

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(15):
    print(f"Epoch {epoch + 1}/15")
    train_one_epoch(model, criterion, optimizer, data_t, "cpu", epoch, 1)
    lr_scheduler.step()

# Inference after retraining
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

Using /root/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_appro1/build.ninja...
Building extension module PyInit_conv2d_appro1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module PyInit_conv2d_appro1...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro1, skipping build step...
Loading extension module PyInit_conv2d_appro1...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro1, skipping build step...
Loading extension module PyInit_conv2d_appro1...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro1, skipping build step...
Loadi

100%|██████████| 2/2 [00:01<00:00,  1.41it/s]
W0222 10:17:21.518513 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:21.519168 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:21.519781 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:21.520325 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:21.520969 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:21.521497 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:21.522929 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:21.523404 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:21.523731 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:21.524072 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:17:21.524416 140419826077888 tensor

conv1.quantizer                         : TensorQuantizer(8bit per-tensor amax=2.1255 calibrator=HistogramCalibrator quant)
conv1.quantizer_w                       : TensorQuantizer(8bit per-tensor amax=0.1513 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.6292 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0620 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer                : TensorQuantizer(8bit per-tensor amax=0.3534 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0380 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.5092 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0354 calibrator=HistogramCalibrator quant)
layer1.1

100%|██████████| 9/9 [00:50<00:00,  5.66s/it]


50.93584382534027
Accuracy of the network on the 10000 test images: 49.0885 %
Epoch 1/15
Epoch: [0]  [ 0/40]  eta: 0:02:20  lr: 0.0001  img/s: 36.71329432445588  loss: 0.3035 (0.3035)  acc1: 91.4062 (91.4062)  acc5: 99.2188 (99.2188)  time: 3.5040  data: 0.0175
Epoch: [0]  [ 1/40]  eta: 0:02:14  lr: 0.0001  img/s: 37.889997310363135  loss: 0.2474 (0.2754)  acc1: 91.4062 (92.5781)  acc5: 99.2188 (99.6094)  time: 3.4499  data: 0.0175
Epoch: [0]  [ 2/40]  eta: 0:02:10  lr: 0.0001  img/s: 38.17494072217618  loss: 0.3035 (0.3270)  acc1: 91.4062 (90.6250)  acc5: 100.0000 (99.7396)  time: 3.4234  data: 0.0175
Epoch: [0]  [ 3/40]  eta: 0:02:05  lr: 0.0001  img/s: 38.426642006625  loss: 0.3035 (0.3362)  acc1: 90.6250 (90.6250)  acc5: 100.0000 (99.8047)  time: 3.4046  data: 0.0174
Epoch: [0]  [ 4/40]  eta: 0:02:02  lr: 0.0001  img/s: 38.459217767760215  loss: 0.3035 (0.3022)  acc1: 91.4062 (91.7188)  acc5: 100.0000 (99.8438)  time: 3.3927  data: 0.0173
Epoch: [0]  [ 5/40]  eta: 0:01:58  lr: 0.00

100%|██████████| 9/9 [00:50<00:00,  5.59s/it]

50.33007787540555
Accuracy of the network on the 10000 test images: 88.9431 %


In [11]:
axx_mult = 'appro2'
model = resnet18(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation
# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)
    
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

# Retraining for 15 epochs
from adapt.references.classification.train import evaluate, train_one_epoch, load_data

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(15):
    print(f"Epoch {epoch + 1}/15")
    train_one_epoch(model, criterion, optimizer, data_t, "cpu", epoch, 1)
    lr_scheduler.step()

# Inference after retraining
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

Using /root/.cache/torch_extensions as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/PyInit_conv2d_appro2...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_appro2/build.ninja...
Building extension module PyInit_conv2d_appro2...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
[1/2] c++ -MMD -MF axx_conv2d.o.d -DTORCH_EXTENSION_NAME=PyInit_conv2d_appro2 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /usr/local/lib/python3.8/dist-packages/torch/include -isystem /usr/local/lib/python3.8/dist-packages/torch/include/torch/csrc/api/include -isystem /usr/local/lib/python3.8/dist-packages/torch/include/TH -isystem /usr/local/lib/python3.8/dist-packages/torch/include/THC -isystem /usr/include/python3.8 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++14 -DAXX_MULT=appro2 -march=na

100%|█████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.15s/it]
W0125 17:39:30.096066 139689568552768 tensor_quantizer.py:173] Disable HistogramCalibrator
W0125 17:39:30.098637 139689568552768 tensor_quantizer.py:173] Disable HistogramCalibrator
W0125 17:39:30.099110 139689568552768 tensor_quantizer.py:173] Disable HistogramCalibrator
W0125 17:39:30.099607 139689568552768 tensor_quantizer.py:173] Disable HistogramCalibrator
W0125 17:39:30.100411 139689568552768 tensor_quantizer.py:173] Disable HistogramCalibrator
W0125 17:39:30.101074 139689568552768 tensor_quantizer.py:173] Disable HistogramCalibrator
W0125 17:39:30.101984 139689568552768 tensor_quantizer.py:173] Disable HistogramCalibrator
W0125 17:39:30.102720 139689568552768 tensor_quantizer.py:173] Disable HistogramCalibrator
W0125 17:39:30.103377 139689568552768 tensor_quantizer.py:173] Disable HistogramCalibrator
W0125 17:39:30.104236 139689568552768 tensor_quantizer.py:17

conv1.quantizer                         : TensorQuantizer(8bit per-tensor amax=2.1255 calibrator=HistogramCalibrator quant)
conv1.quantizer_w                       : TensorQuantizer(8bit per-tensor amax=0.1513 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.6268 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0620 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer                : TensorQuantizer(8bit per-tensor amax=0.3531 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0380 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.5139 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0354 calibrator=HistogramCalibrator quant)
layer1.1

100%|█████████████████████████████████████████████████████████████████████████████████████| 9/9 [03:40<00:00, 24.54s/it]


220.8976204619994
Accuracy of the network on the 10000 test images: 10.0586 %
Epoch 1/15
Epoch: [0]  [ 0/40]  eta: 0:07:00  lr: 0.0001  img/s: 12.224135996042708  loss: 2.3627 (2.3627)  acc1: 11.7188 (11.7188)  acc5: 53.1250 (53.1250)  time: 10.5118  data: 0.0407
Epoch: [0]  [ 1/40]  eta: 0:07:04  lr: 0.0001  img/s: 11.415617568296637  loss: 2.3627 (2.3917)  acc1: 9.3750 (10.5469)  acc5: 43.7500 (48.4375)  time: 10.8843  data: 0.0424
Epoch: [0]  [ 2/40]  eta: 0:06:56  lr: 0.0001  img/s: 11.525680754020016  loss: 2.3784 (2.3873)  acc1: 10.9375 (10.6771)  acc5: 53.1250 (50.0000)  time: 10.9719  data: 0.0421
Epoch: [0]  [ 3/40]  eta: 0:06:45  lr: 0.0001  img/s: 11.763772834300594  loss: 2.3784 (2.3912)  acc1: 10.1562 (10.5469)  acc5: 43.7500 (48.4375)  time: 10.9606  data: 0.0430
Epoch: [0]  [ 4/40]  eta: 0:06:32  lr: 0.0001  img/s: 11.990997129889012  loss: 2.4030 (2.3965)  acc1: 10.1562 (10.0000)  acc5: 50.7812 (48.9062)  time: 10.9133  data: 0.0443
Epoch: [0]  [ 5/40]  eta: 0:06:22  lr

100%|█████████████████████████████████████████████████████████████████████████████████████| 9/9 [03:20<00:00, 22.28s/it]

200.53065981100008
Accuracy of the network on the 10000 test images: 9.9718 %


In [9]:
axx_mult = 'appro3'
model = resnet18(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation
# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)
    
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

# Retraining for 15 epochs
from adapt.references.classification.train import evaluate, train_one_epoch, load_data

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(15):
    print(f"Epoch {epoch + 1}/15")
    train_one_epoch(model, criterion, optimizer, data_t, "cpu", epoch, 1)
    lr_scheduler.step()

# Inference after retraining
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

Using /root/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_appro3/build.ninja...
Building extension module PyInit_conv2d_appro3...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module PyInit_conv2d_appro3...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro3, skipping build step...
Loading extension module PyInit_conv2d_appro3...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro3, skipping build step...
Loading extension module PyInit_conv2d_appro3...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro3, skipping build step...
Loadi

100%|██████████| 2/2 [00:01<00:00,  1.44it/s]
W0222 10:52:35.969357 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:52:35.969964 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:52:35.970372 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:52:35.970776 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:52:35.971172 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:52:35.971542 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:52:35.971900 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:52:35.972275 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:52:35.972671 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:52:35.973087 140419826077888 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 10:52:35.973517 140419826077888 tensor

conv1.quantizer                         : TensorQuantizer(8bit per-tensor amax=2.1255 calibrator=HistogramCalibrator quant)
conv1.quantizer_w                       : TensorQuantizer(8bit per-tensor amax=0.1513 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.6280 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0620 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer                : TensorQuantizer(8bit per-tensor amax=0.3550 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0380 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.5166 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0354 calibrator=HistogramCalibrator quant)
layer1.1

100%|██████████| 9/9 [00:50<00:00,  5.59s/it]


50.346718752756715
Accuracy of the network on the 10000 test images: 10.0586 %
Epoch 1/15
Epoch: [0]  [ 0/40]  eta: 0:02:16  lr: 0.0001  img/s: 37.63622925540092  loss: 0.4635 (0.4635)  acc1: 89.0625 (89.0625)  acc5: 100.0000 (100.0000)  time: 3.4194  data: 0.0184
Epoch: [0]  [ 1/40]  eta: 0:02:14  lr: 0.0001  img/s: 37.24415193856637  loss: 0.4635 (0.4836)  acc1: 84.3750 (86.7188)  acc5: 99.2188 (99.6094)  time: 3.4392  data: 0.0203
Epoch: [0]  [ 2/40]  eta: 0:02:10  lr: 0.0001  img/s: 37.432335383449825  loss: 0.5036 (0.5463)  acc1: 84.3750 (84.6354)  acc5: 99.2188 (99.4792)  time: 3.4385  data: 0.0194
Epoch: [0]  [ 3/40]  eta: 0:02:07  lr: 0.0001  img/s: 36.911008367956896  loss: 0.5036 (0.5567)  acc1: 82.0312 (83.9844)  acc5: 99.2188 (99.2188)  time: 3.4502  data: 0.0189
Epoch: [0]  [ 4/40]  eta: 0:02:04  lr: 0.0001  img/s: 36.93537376612553  loss: 0.5036 (0.5434)  acc1: 82.0312 (83.5938)  acc5: 99.2188 (99.2188)  time: 3.4568  data: 0.0186
Epoch: [0]  [ 5/40]  eta: 0:02:01  lr: 0.

100%|██████████| 9/9 [00:50<00:00,  5.64s/it]

50.72582599334419
Accuracy of the network on the 10000 test images: 85.5143 %
